In [44]:
# pip install ta
# pip install pandas_ta

import pandas as pd
import yahoo_fin.stock_info as si
from ta import add_all_ta_features
from ta.momentum import RSIIndicator
from ta.trend import macd

ticker = 'TSLA'


def macd_rsi():
    # pull data from Yahoo Finance
    data = si.get_data("aapl")
    # add technical analysis features
    data = add_all_ta_features(data, open="open", high="high", low="low", close="adjclose", volume="volume")
    # print(data)
    rsi_21 = RSIIndicator(close=data.adjclose, window=21)
    data["rsi_21"] = rsi_21.rsi()
    data["macd"] = macd(data.adjclose, window_slow=26, window_fast=12)
    print(data)


# https://algotrading101.com/learn/yahoo-finance-api-guide/
# get_data(ticker, start_date=None, end_date=None, index_as_date=True, interval="1d")

# amazon_weekly = get_data(ticker, start_date="2022-02-02", end_date="2022-04-20", index_as_date=True, interval="1wk")
# print(amazon_weekly)

quote_table = si.get_quote_table(ticker, dict_result=False)
# print(quote_table)

value = quote_table[quote_table["attribute"] == "EPS (TTM)"]["value"]
is_positive_eps = value > 5
print(is_positive_eps.bool())


market_cap = quote_table[quote_table["attribute"] == "Market Cap"]["value"]
value = market_cap.str.contains('B|T', regex=True)
print(value.bool())

pe_ratio = quote_table[quote_table["attribute"] == "PE Ratio (TTM)"]["value"]
print(pe_ratio > 1)

avg_volume = quote_table[quote_table["attribute"] == "Avg. Volume"]["value"]
print(avg_volume)

# if (quote_table["Attribute"] == "EPS (TTM)"):
#     print('Great')

# Divident
# quote_table = si.get_quote_table("aapl")
# quote_table["Forward Dividend & Yield"]
# print(quote_table)

# cash_flow_statement = si.get_cash_flow(ticker)
# print(cash_flow_statement["breakdown"])
# cash_flow_statement.to_csv( "cash_flow_statement.csv", date_format='%Y-%m-%d')



False
True
13    True
Name: value, dtype: bool
3    25681272.0
Name: value, dtype: object


In [50]:

# get list of Dow tickers
# dow_list = si.tickers_nasdaq()
dow_list = ["AAPL","MSFT","AMZN","TSLA","GOOGL","NVDA","FB","COST","AVGO","PEP","CMCSA","CSCO","ADBE","INTC","TMUS","TXN","QCOM","AMD","NFLX","AMGN","INTU","HON","PYPL","ISRG","AMAT","ADP","CHTR","SBUX","BKNG","MDLZ"]

dow_stats = {}
for ticker in dow_list:
    try:
        temp = si.get_stats_valuation(ticker)

        quote_table = si.get_quote_table(ticker, dict_result=False)
        # print(quote_table)
        eps_value = quote_table[quote_table["attribute"] == "EPS (TTM)"]["value"]
        is_positive_eps = eps_value > 1

        market_cap = quote_table[quote_table["attribute"] == "Market Cap"]["value"]
        is_market_cap_high = market_cap.str.contains('B|T', regex=True)

        pe_ratio = quote_table[quote_table["attribute"] == "PE Ratio (TTM)"]["value"]
        is_positive_pe = pe_ratio > 1

        avg_volume = quote_table[quote_table["attribute"] == "Avg. Volume"]["value"]
        has_good_volume = avg_volume > 100000

        # print(f"Ticker: {ticker}, {is_positive_eps.bool()}, {is_market_cap_high.bool()}, {is_positive_pe.bool()}, {has_good_volume.bool()}")

        if is_positive_eps.bool() and \
                is_market_cap_high.bool() and \
                is_positive_pe.bool() and \
                has_good_volume.bool():
            print(ticker)
            temp = temp.iloc[:, :2]
            temp.columns = ["Attribute", "Recent"]
            dow_stats[ticker] = temp
    except Exception as ex:
        print(ex)

print(len(dow_stats))


AAPL
MSFT
AMZN
TSLA
html5lib not found, please install it
GOOGL
NVDA
FB
COST
AVGO
PEP
CMCSA
CSCO
ADBE
INTC
TMUS
TXN
QCOM
AMD
NFLX
AMGN
INTU
HON
PYPL
ISRG
AMAT
ADP
CHTR
SBUX
BKNG
MDLZ
30


In [51]:

combined_stats = pd.concat(dow_stats)
combined_stats = combined_stats.reset_index()

del combined_stats["level_1"]
# update column names
combined_stats.columns = ["Ticker", "Attribute", "Recent"]
print(combined_stats)

pe_ratios = combined_stats[combined_stats["Attribute"] == "Trailing P/E"].reset_index()
print(pe_ratios)

pe_ratios_sorted = pe_ratios.sort_values('Recent', ascending=False)
print(pe_ratios_sorted)

# macd_rsi()

    Ticker                  Attribute Recent
0     AAPL      Market Cap (intraday)  2.70T
1     AAPL           Enterprise Value  2.76T
2     AAPL               Trailing P/E  27.41
3     AAPL                Forward P/E  27.47
4     AAPL  PEG Ratio (5 yr expected)   3.19
..     ...                        ...    ...
265   MDLZ  PEG Ratio (5 yr expected)   3.35
266   MDLZ          Price/Sales (ttm)   3.13
267   MDLZ           Price/Book (mrq)   3.12
268   MDLZ   Enterprise Value/Revenue   3.64
269   MDLZ    Enterprise Value/EBITDA  17.88

[270 rows x 3 columns]
    index Ticker     Attribute  Recent
0       2   AAPL  Trailing P/E   27.41
1      11   MSFT  Trailing P/E   29.80
2      20   AMZN  Trailing P/E   46.82
3      29   TSLA  Trailing P/E  201.02
4      38  GOOGL  Trailing P/E   22.59
5      47   NVDA  Trailing P/E   52.42
6      56     FB  Trailing P/E   15.26
7      65   COST  Trailing P/E   47.57
8      74   AVGO  Trailing P/E   32.72
9      83    PEP  Trailing P/E   31.31
10     